In [32]:
import re
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os, glob
import time
import pandas as pd
import numpy as np
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)

C:\Users\jasak\AppData\Local\Temp\ipykernel_12296\159072978.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


In [2]:
seznam_supertypov = ["Basic", "Legendary", "Ongoing", "Snow", "World"]

vzorec_bloka = re.compile(
    r'<td><mtg-set-icon.*?'
    r'</tr',
    flags=re.DOTALL
)

vzorec_karte = re.compile(
    r'<td><mtg-set-icon.*?>.*?<i _ngcontent.*? class="ss ss-fw ss-(?P<set>.+?)\b( ss-.*?)?"></i><!----></mtg-set-icon>'
    r'<a href="/prints/(?P<id_karte>\d*).*?">'
    r'(?P<ime>.*?)'
    r'</a>.*?</td>.*?<td> (?P<redkost>.*?) </td>'
    r'.*?<td class="text-end">'
    r' (?P<povprecna_cena>.*?) </td>(<\!---->)+(<td class="text-end">.*?</td>.*?<td class="text-end"> (?P<povprecna_cena_foil>.*?) </td>.*?<td class="text-end">.*?</td>)?',  
    flags=re.DOTALL
    )

def izloci_podatke_o_kartah(blok):
    karta = vzorec_karte.search(blok).groupdict()
    karta['id_karte'] = int(karta['id_karte'])
    karta['ime'] = karta['ime'].strip()
    karta['set'] = karta['set'].upper()
    karta['redkost'] = karta["redkost"].strip()
    karta['povprecna_cena'] = karta["povprecna_cena"]
    karta['povprecna_cena_foil'] = karta["povprecna_cena_foil"]

    return karta




def izloci_podatke_o_specificni_karti(vsebina):
    """
    To bo izločilo vse dodatne podatke o karti, ki jih bomo našli na naslovu .../prints/<id_karte>
    """
    pass

def pridobi_ustrezno_ime_lokalne_datoteke(st_strani):
    return f"Podatki o kartah/Podatki iz setov/Karte iz seta st. {st_strani}.html"

In [ ]:


# PRVI_SET = 1
# ZADNJI_SET = 1300

# #indeksi_setov_ki_obstajajo = []

# for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
#     url = (
#         f'https://www.mtgstocks.com/sets/{st_strani}' 
#     )
#     print(f"\n Zajemam {url}\n")
#     driver.get(url)
#     time.sleep(1)
    
    
#     vsebina = driver.page_source
#     naslov_strani = driver.title
    
#     stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
#     st_besed_Market_na_strani = vsebina.count(">Market<")
    
#     print("Naslov strani:", naslov_strani)
    
#     if naslov_strani in ["File not found - MTGStocks"]:
#         print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
#     else:
#         print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
#         if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
#             continue
#         else:
#             with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w', encoding = "utf-8") as dat:
#                 driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
#                 for podstran in range(1, stevilo_podstrani_s_kartami + 1):
#                     driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
#                     time.sleep(1)
#                     print(f"Pobrana {podstran}. podstran")
#                     dat.write(driver.page_source)
  

Koda za zapisovanje kart iz lokalnih datotek v csv in json format.


In [16]:
debug_mode_za_branje_iz_lokalnih_datotek = False
path = 'Podatki o kartah/Podatki iz setov'
karte = []

# # Najprej poradiramo csv in json datoteki
with open("karte.csv", "w", encoding = "utf-8") as dat:
    pass

with open("karte.json", "w", encoding = "utf-8") as dat:
    pass


for filename in glob.glob(os.path.join(path, '*.html')):
    # Tukaj iz nekega razloga ne deluje utf-8
    with open(filename, 'r', encoding = "latin1") as f:

        stetje = 0
        print(f"Berem podatke kart iz {filename}")
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            if debug_mode_za_branje_iz_lokalnih_datotek:
                print(stetje)
                print(blok.group(0))
                print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izloci_podatke_o_kartah(blok.group(0)))

with open("karte.json", "a", encoding = "utf-8") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a", encoding = "utf-8") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 10.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 100.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1008.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1009.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 101.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1011.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 102.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 103.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 104.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1041.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 105.html
Ber

Zdaj se znebimo kart, ki so tokeni ali basic landi.
Prav tako bi se radi znebili kart, ki imajo tako ceno kot ceno foila ničelno. (Torej da lahko karte primerjamo po vsaj eni komponenti).
Pri tem je vredno omeniti, da imajo karte, ki so najbolj zaželene, tudi najbolj popolne podatke, torej jih s tem ne bomo izgubili.

Poleg teh podatkov bom pridobil še podatke s spletne strani "https://scryfall.com/sets", kjer lahko najdem datum izida posameznih kart.


In [42]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki_o_kartah = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki_o_kartah = okrajsani_podatki_o_kartah[~(okrajsani_podatki_o_kartah.povprecna_cena.isnull() & okrajsani_podatki_o_kartah.povprecna_cena_foil.isnull())]

# Nekateri seti so na strani MTGStocks navedeni napačno! Zato jih je treba nekoliko popraviti
# Takih setov je malo, zato sem jih popravil na roko. Poleg tega sta dva seta iz tehle zelo zelo pomembna, zato se mi je zdelo
# popravljanje na roko v tem primeru upravičeno. (podatki so dovolj nezanemarljivi).
# PFNM je problem, saj je veliko kart, ki so v bistvu iz 20 različnih setov, pod enim imenom!
# Podobno PMTG1
okrajsani_podatki_o_kartah = okrajsani_podatki_o_kartah.replace({'set': {"VAN" : "PVAN", "SLDC": "PHED", "XCLE": "CED", "XICE": "CEI", "RMB1": "FMB1", "XDND" : "AFR", "2U": "2ED", "1E": "LEA", "PO2": "P02"}})


okrajsani_indeksi = okrajsani_podatki_o_kartah.id_karte # To so indeksi, s katerimi bomo pognali ustrezne url-je

raznoliki_seti = okrajsani_podatki_o_kartah["set"].unique()

print(okrajsani_podatki_o_kartah)
print(raznoliki_seti)


       id_karte               ime  set   redkost povprecna_cena  \
0         19748     Cyclonic Rift  RTR      Rare         $38.00   
1         19934   Utvara Hellkite  RTR    Mythic         $19.53   
2           360       Blood Crypt  RTR      Rare         $19.99   
3           404       Steam Vents  RTR      Rare         $18.63   
4           395    Overgrown Tomb  RTR      Rare         $13.99   
...         ...               ...  ...       ...            ...   
47532     18337  Dregscape Zombie  ARC    Common          $0.25   
47533     18380        Juggernaut  ARC  Uncommon          $0.25   
47534     18421           Plummet  ARC    Common          $0.25   
47535     18348          Fertilid  ARC    Common          $0.38   
47536     18434  Sanctum Gargoyle  ARC    Common          $0.24   

      povprecna_cena_foil  
0                  $57.99  
1                  $57.44  
2                  $34.37  
3                  $38.61  
4                  $25.72  
...                   ...  

In [43]:
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)
url_do_strani_setov = "https://scryfall.com/sets"

driver.get(url_do_strani_setov)
time.sleep(1)
vsebina_strani_setov = driver.page_source
driver.close()

slovarji_posameznih_setov = []
problematicni_seti = ["PFNM", "PMTG1", "DD3", "PARL2"]

def izloci_podatke_o_setih(vsebina, vzorec, posamezen_set):
    if posamezen_set in problematicni_seti:
        # Ker je tako malo setov, jih lahko popravimo na roko
        # Tega nisem mogels storiti drugače, saj so bile napake na strežniku, podatki pa zelo pomembni
        if posamezen_set == "DD3":
            return {"set": posamezen_set, "polno_ime": "Duel Decks: Anthology", "st_kart": podatki_o_kartah["set"].value_counts()[posamezen_set], "datum_izida": "2014-12-05"}
        elif posamezen_set == "PFNM":
            return {"set": posamezen_set, "polno_ime": "Friday Night Magic: Promos", "st_kart": podatki_o_kartah["set"].value_counts()[posamezen_set], "datum_izida": None}
        elif posamezen_set == "PMTG1":
            return {"set": posamezen_set, "polno_ime": "Promos", "st_kart": podatki_o_kartah["set"].value_counts()[posamezen_set], "datum_izida": None}
        elif posamezen_set == "PARL2":
            return {"set": posamezen_set, "polno_ime": "Arena Promos", "st_kart": podatki_o_kartah["set"].value_counts()[posamezen_set], "datum_izida": None}
    
    set = vzorec.search(vsebina).groupdict()
    set["set"] = posamezen_set
    set['polno_ime'] = set['polno_ime'].strip()
    set['st_kart'] = int(set['st_kart'])
    set['datum_izida'] = set["datum_izida"] # tukaj bi se morda dalo še kaj dodati

    return set

with open("podatki_o_specificnih_setih.csv", "w", encoding="utf-8") as csvfile:
    pass

with open("podatki_o_specificnih_setih.csv", "a", encoding="utf-8") as csvfile:
    
    writer = csv.DictWriter(csvfile, [
            "set",
            "polno_ime",
            "st_kart",
            "datum_izida",
        ])
    
    for posamezen_set in raznoliki_seti:
        vzorec_za_podatke_o_setu = re.compile(
            r'</svg>(?P<polno_ime>.*?) <small>' + posamezen_set + r'.*?">(?P<st_kart>.*?)<.*?">(?P<datum_izida>.*?)<',
            flags=re.DOTALL
        )
        
        print("Shranjen", posamezen_set, end = " ")
        
        try:
            slovarji_posameznih_setov.append(izloci_podatke_o_setih(vsebina_strani_setov, vzorec_za_podatke_o_setu, posamezen_set))
        except Exception as e:
            print("Nekaj ni bilo ok pri setu", posamezen_set)
            print(e)
    writer.writeheader()
    writer.writerows(slovarji_posameznih_setov)
    
    
    with open("seti.json", "a", encoding = "utf-8") as dat:
        json.dump(karte, dat, indent=4, ensure_ascii=False)

C:\Users\jasak\AppData\Local\Temp\ipykernel_12296\2611835630.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


Shranjen RTR Shranjen ZEN Shranjen BRB Shranjen SNC Shranjen NCC Shranjen BTD Shranjen CHR Shranjen CMD Shranjen HOP Shranjen CSP Shranjen PC2 Shranjen UGL Shranjen UNH Shranjen 2X2 Shranjen PVAN Shranjen ARB Shranjen GTC Shranjen CLB Shranjen DGM Shranjen DDK Shranjen PFNM Shranjen DMU Shranjen DMC Shranjen GN3 Shranjen 40K Shranjen PHED Shranjen PMTG1 Shranjen BRO Shranjen CON Shranjen ALA Shranjen EVE Shranjen SHM Shranjen MOR Shranjen LRW Shranjen FUT Shranjen PLC Shranjen AVR Shranjen TSP Shranjen MMA Shranjen M14 Shranjen THS Shranjen DDL Shranjen C13 Shranjen BNG Shranjen DIS Shranjen DDM Shranjen JOU Shranjen CNS Shranjen MD1 Shranjen M15 Shranjen DDN Shranjen KTK Shranjen GPT Shranjen C14 Shranjen DD3 Shranjen FRF Shranjen DDO Shranjen DTK Shranjen MM2 Shranjen ORI Shranjen RAV Shranjen DDP Shranjen BFZ Shranjen C15 Shranjen OGW Shranjen DDQ Shranjen DKM Shranjen SOI Shranjen SOK Shranjen EMA Shranjen EMN Shranjen CN2 Shranjen KLD Shranjen C16 Shranjen AER Shranjen PCA Shranje

Zdaj pa pogledamo vsako posamezno karto. Zato bomo naredili posamezne vzorce, ki nam bodo pomagali pri parsanju.
Nesmiselno je imeti samo en vzorec, saj je to samo logistična nočna mora.


In [18]:
vzorec_povprecja_eu = re.compile(
    r'Europe avg </p> (?P<povprecje_eu>.*?) <',
    flags=re.DOTALL
)

vzorec_cardtypea = re.compile(
    r'class="card-oracle-item".*?</i> (?P<cardtype>.*?) <',
    flags=re.DOTALL
)

vzorec_oracle_texta = re.compile(
    r'<.*?class="card-oracle-item card-oracle-text">(?P<oracle_text>.*?)</p>',
    flags=re.DOTALL
)

vzorec_all_time_low = re.compile(
    r'All Time Low </p><h3.*?>(?P<all_time_low>.*?)</h3><h5.*?> (?P<datum_all_time_low>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_all_time_high = re.compile(
    r'All Time High </p><h3.*?>(?P<all_time_high>.*?)</h3><h5.*?> (?P<datum_all_time_high>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_reserved_list = re.compile(
    r'Reserved list\?</p><h3.*?>(?P<reserved_list>.*?)</h3>',
    flags=re.DOTALL
)

vzorec_za_popravo_oracle_texta_notranje_znacke = re.compile(
    r'<i class="ms ms-cost ms-(.*?)"></i>',
    flags=re.DOTALL
)
vzorec_za_popravo_oracle_texta_zunanje_znacke = re.compile(
    r'<i>\((.*)\)</i>',
    flags=re.DOTALL
)
vzorec_za_popravo_oracle_texta_planeswalker_uptake = re.compile(
    r'<i class="ms ms-loyalty-(\d*) ms-loyalty-up"></i>',
    flags=re.DOTALL
)

vzorec_za_popravo_oracle_texta_planeswalker_downtake = re.compile(
    r'<i class="ms ms-loyalty-(\d*) ms-loyalty-down"></i>',
    flags=re.DOTALL
)


def doloci_super_sub_in_cardtype(niz):
    niz = niz.removesuffix(", ")
    supertype, subtype, cardtype = [], [], []
    for suprtype in seznam_supertypov:
        if suprtype in niz:
            supertype.append(suprtype)
            niz = niz.replace(f"{suprtype} ","")
    if "-" in niz or "—" in niz:
        niz = niz.replace("—","-")
        str_cardtype, str_subtype = niz.split(" - ")
        cardtype = str_cardtype.split(" ")
        subtype = str_subtype.split(" ")
    elif " " in niz:
        cardtype = niz.split(" ")
    else:
        cardtype.append(niz)
        
    return supertype, cardtype, subtype

def pretvori_findall_seznam_v_singleton(seznam):
    if len(seznam) == 1:
        return seznam[0]
    else:
        return None

def izlusci_podatke_iz_glavnine(niz):
    povprecje_eu = re.findall(vzorec_povprecja_eu, niz)
    cardtype = re.findall(vzorec_cardtypea, niz)
    oracle_text = re.findall(vzorec_oracle_texta, niz)
    reserved_list = re.findall(vzorec_reserved_list, niz)
    najdi_vse_low = re.findall(vzorec_all_time_low, niz)
    najdi_vse_high = re.findall(vzorec_all_time_high, niz)
    
    povprecje_eu, cardtype, oracle_text, reserved_list = map(pretvori_findall_seznam_v_singleton, [povprecje_eu, cardtype, oracle_text, reserved_list])
    
    if cardtype:
        print(cardtype)
        
        supertype, cardtype, subtype = doloci_super_sub_in_cardtype(cardtype)
    else:
        supertype, subtype = [], []
    
    if oracle_text:
        oracle_text = re.sub(vzorec_za_popravo_oracle_texta_notranje_znacke, r"(\1)", oracle_text)
        oracle_text = re.sub(vzorec_za_popravo_oracle_texta_zunanje_znacke, r"\1", oracle_text)
        oracle_text = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_uptake, r"(\+\1)", oracle_text)
        oracle_text = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_downtake, r"(\-\1)", oracle_text)
        oracle_text = oracle_text.replace('""(t)',"(tap)").replace('(t)',"(tap)") # To je zaradi buga na spletni strani
    
    if len(najdi_vse_low) == 1:
        if len(najdi_vse_low[0]) == 2:
            all_time_low, all_time_low_datum = najdi_vse_low[0][0], najdi_vse_low[0][1]
    else:
        all_time_low, all_time_low_datum = None, None
    if len(najdi_vse_high) == 1:
        if len(najdi_vse_high[0]) == 2:
            all_time_high, all_time_high_datum = najdi_vse_high[0][0], najdi_vse_high[0][1]
    else:
        all_time_high, all_time_high_datum = None, None
    return {"povprecje_eu": povprecje_eu, "supertype": supertype, "cardtype": cardtype, "subtype": subtype, "oracle_text": oracle_text, "reserved_list":reserved_list, "all_time_low": all_time_low, "all_time_low_datum": all_time_low_datum, "all_time_high": all_time_high, "all_time_high_datum": all_time_high_datum}



In [6]:
vzorec_za_manacost = re.compile(r'" class="ms ms-cost ms-(?P<mana_cost>.*?)["| ]',
                                flags=re.DOTALL)

def izlusci_podatke_manacosta_in_barve(niz):
    seznam_simbolov = re.findall(vzorec_za_manacost, niz)
    cmc = 0
    manacost = ""
    barva = ""
    for simbol in seznam_simbolov:
        manacost += simbol
        if simbol.isnumeric():
            cmc += int(simbol)
        elif simbol in "wubrg":
            cmc += 1
            if simbol not in barva:
                barva += simbol
        else:
            if simbol not in "{(p)}x":
                cmc += 1
            if simbol not in "{(p)}c":
                print("Program je našel nenavaden simbol", simbol)
    
    # To je majhen popravek za karte s hybrid manacostom
    # Potreben je, ker je to bug na strani MTGSrocks (narobno zapisuje hybrid cost)
    cmc -= barva.count("(") + barva.count("{")
    
    if barva == "":
        barva += "c"
    if manacost == "":
        manacost += "0"
    return {"cmc": cmc, "manacost": manacost, "barva": barva}

with open("testni_zajcek.txt", "r", encoding = "utf-8") as testna_datoteka:
    vsebina = testna_datoteka.read()
    
glavnina = izlusci_podatke_iz_glavnine(vsebina)
manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
manacost_in_barve.update(glavnina)
print(manacost_in_barve["oracle_text"])


Legendary Planeswalker, 
(\+2): Look at the top card of target player's library. You may put that card on the bottom of that player's library.
0: Draw three cards, then put two cards from your hand on top of your library in any order.
(\-1): Return target creature to its owner's hand.
(\-12): Exile all cards from target player's library, then that player shuffles his or her hand into his or her library.


#Blok, ki požene branje vsake posamezne karte

In [7]:
print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))

# To je toliko da poradiramo
with open("specificne_lastnosti_kart.csv", "w", encoding = "utf-8") as csvfile:
    pass

with open("specificne_lastnosti_kart.csv", "a", encoding = "utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, [
        "id_karte",
        "cmc",
        "manacost",
        "barva",
        "povprecje_eu",
        "supertype",
        "cardtype",
        "subtype",
        "oracle_text",
        "reserved_list",
        "all_time_low",
        "all_time_low_datum",
        "all_time_high",
        "all_time_high_datum",
    ])
    
    podatki_o_kartah = []

    for indeks_printa in okrajsani_indeksi[:100]:
        url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
        print(f"Zajemanje printa karte št. {indeks_printa} s strani {url}")
        # with open(f"Podatki o kartah/Podatki o specificnih kartah/Podatki o karti st. {indeks_printa}.html", "w", encoding="utf-8") as dat:
        try:
            driver.get(url)
            time.sleep(1)
            vsebina = driver.page_source
            
            glava_slovarja = {"id_karte": indeks_printa}
            manacost_in_barve = izlusci_podatke_manacosta_in_barve(vsebina)
            glavnina = izlusci_podatke_iz_glavnine(vsebina)
            glava_slovarja.update(manacost_in_barve)
            glava_slovarja.update(glavnina)
            print(glava_slovarja, "\n")
            podatki_o_kartah.append(glava_slovarja)
            
        except Exception as e:
            print(f"Prišlo je do napake pri setu {indeks_printa}")
            print(e)
            
    writer.writeheader()
    writer.writerows(podatki_o_kartah)
        

Število vseh indeksov kart, ki jih imamo: 129173
Zajemanje printa karte št. 19748 s strani https://www.mtgstocks.com/prints/19748


C:\Users\jasak\AppData\Local\Temp\ipykernel_12296\4103053668.py:27: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for indeks_printa in okrajsani_indeksi[:100]:


Instant
{'id_karte': '19748', 'cmc': 2, 'manacost': '1u', 'barva': 'u', 'povprecje_eu': '€24.74', 'supertype': [], 'cardtype': ['Instant'], 'subtype': [], 'oracle_text': 'Return target nonland permanent you don\'t control to its owner\'s hand.\nOverload (6)(u) You may cast this spell for its overload cost. If you do, change its text by replacing all instances of "target" with "each."', 'reserved_list': 'No', 'all_time_low': '$1.79', 'all_time_low_datum': 'Apr 18, 2013', 'all_time_high': '$40.10', 'all_time_high_datum': 'Oct 11, 2022'} 

Zajemanje printa karte št. 19934 s strani https://www.mtgstocks.com/prints/19934
Creature - Dragon, 
{'id_karte': '19934', 'cmc': 8, 'manacost': '6rr', 'barva': 'r', 'povprecje_eu': '€12.61', 'supertype': [], 'cardtype': ['Creature'], 'subtype': ['Dragon'], 'oracle_text': 'Flying\nWhenever a Dragon you control attacks, put a 6/6 red Dragon creature token with flying onto the battlefield.', 'reserved_list': 'No', 'all_time_low': '$1.38', 'all_time_low_da

# Spodaj se nahajajo testni bloki

In [ ]:
string_za_poskus = ''
print(re.findall(vzorec_za_manacost, string_za_poskus))

print(izlusci_podatke_manacosta_in_barve(string_za_poskus))

In [ ]:
testni_string = '<i class="ms ms-loyalty-1 ms-loyalty-up"></i>: Until your next turn, whenever a creature deals combat damage to Vraska the Unseen, destroy that creature.\n<i class="ms ms-loyalty-3 ms-loyalty-down"></i>: Destroy target nonland permanent.\n<i class="ms ms-loyalty-7 ms-loyalty-down"></i>: Put three 1/1 black Assassin creature tokens onto the battlefield with "Whenever this creature deals combat damage to a player, that player loses the game."'
with open("testni_zajcek.txt", "r", encoding = "utf-8") as testna_datoteka:
    testni_string = testna_datoteka.read()

popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_notranje_znacke, r"(\1)", testni_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_zunanje_znacke, r"\1", popravljen_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_uptake, r"(+\1)", popravljen_string)
popravljen_string = re.sub(vzorec_za_popravo_oracle_texta_planeswalker_downtake, r"(-\1)", popravljen_string)
popravljen_string = popravljen_string.replace('""(t)',"(tap)") # To je zaradi buga na spletni strani
print(popravljen_string)

In [11]:
testni_string = "Legendary Planeswalker — Jace, "

def doloci_super_sub_in_cardtype(niz):
    niz = niz.removesuffix(", ")
    supertype, subtype, cardtype = [], [], []
    for suprtype in seznam_supertypov:
        if suprtype in niz:
            supertype.append(suprtype)
            niz = niz.replace(f"{suprtype} ","")
    if "-" in niz or "—" in niz:
        niz = niz.replace("—","-")
        str_cardtype, str_subtype = niz.split(" - ")
        cardtype = [typ.strip() for typ in str_cardtype.split(" ") if typ.strip() != ""]
        subtype = [typ.strip() for typ in str_subtype.split(" ") if typ.strip() != ""]
    elif " " in niz:
        cardtype = [typ.strip() for typ in niz.split(" ") if typ.strip() != ""]
    else:
        cardtype.append(niz.strip())
        
    return supertype, cardtype, subtype

print(doloci_super_sub_in_cardtype(testni_string))

    

(['Legendary'], ['Planeswalker'], ['Jace'])
